In [1]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

import pickle_utils as pu
import numpy as np
from scipy.ndimage import convolve1d
import tensorflow as tf

In [2]:
X, y = pu.load('clean_ventilation/interpolation/cat_0.pkl.gz')
# The last item in `total_counts` is the number of missing
total_counts = pu.load('clean_ventilation/interpolation/counts_cat_0.pkl.gz')[:-1]
len_t = max(t for _, t in X)+1
N_cats = max(map(lambda x: x[0], X))+1
counts = np.zeros([N_cats, N_cats, len_t], dtype=np.float)

for (cat, t), c in zip(X, y):
    counts[cat,c,t] += 1
counts.shape, total_counts.shape

((6, 6, 534), (6,))

We create the training and test sets by drawing some examples at random

In [4]:
counts_ = np.reshape(counts, [-1])
total_examples = np.sum(counts_)
counts_ = counts_ / total_examples
test_set = np.random.multinomial(int(total_examples*.1), pvals=counts_)
test_set = np.reshape(test_set, [N_cats, N_cats, len_t])
training_set = counts - test_set
assert np.all(training_set >= 0), "Try another time until it works"

In [12]:
pu.dump((training_set, test_set), "training_test.pkl.gz")

Now we normalize `counts` to produce `p`, and plot it.

In [5]:
norm = np.sum(counts+1, axis=1, keepdims=True)
norm = np.tile(norm, [1,N_cats,1])
counts[norm==0] += 1
counts[norm==0] += counts.shape[0]
p = counts/norm

plots = []
for i in range(p.shape[1]):
    plots.append({'y': p[0, i, :]})
    
iplot(plots)

We create a convolutional filter as long as double the time series, and populate it with a smoothing function.

In [6]:
np_filter = np.zeros([len_t*2+1], dtype=np.float)
np_filter[:len_t] = np.arange(len_t, 0, -1)
np_filter[len_t+1:] = np.arange(len_t)+1
np_filter = np.reshape(np_filter, [len_t*2+1, 1, 1])

np_smoothing = np.reshape(total_counts, [1,N_cats,1])/np.sum(total_counts)

with tf.variable_scope("cat_0d"):
    tf_filter_i = tf.constant(np_filter, dtype=tf.float32)
    scale = tf.get_variable("scale", shape=[], dtype=tf.float32,
                            trainable=True,
                            initializer=tf.constant_initializer(-1))
    tf_filter = tf.exp(tf_filter_i*scale)
    data = tf.constant(training_set, dtype=tf.float32)
    data_ = tf.pad(tf.reshape(data, [N_cats*N_cats, len_t, 1]),
                  [(0,0), (len_t, len_t), (0, 0)]) 
    ns_ = tf.nn.conv1d(data_, tf_filter, stride=1, padding='VALID')
    ns = tf.reshape(ns_, [N_cats, N_cats, len_t])+np_smoothing
    ps = ns / tf.reduce_sum(ns, axis=1, keep_dims=True)
    log_likelihood = tf.reduce_sum(test_set*tf.log(ps))

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    probas = sess.run(ps)

This is the length scale without being optimised

In [8]:
plots = []
for i in range(probas.shape[1]):
    plots.append({'y': probas[0, i, :]})
    
iplot(plots)

In [9]:
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(-log_likelihood)
patience = 5
times_waiting = 0
prev_ll = -np.inf
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100000):
        sess.run(train_op)
        if i%100 == 0:
            _, ll, s = sess.run([train_op, log_likelihood, scale])
            print("step", i, ll, ", scale =", s)
            if ll > prev_ll:
                times_waiting = 0
            else:
                times_waiting += 1
                if times_waiting > patience:
                    break
            prev_ll = ll
        
    s, probas = sess.run([scale, ps])

step 0 -272.79 , scale = -0.998
step 100 -271.349 , scale = -0.896392
step 200 -269.849 , scale = -0.793593
step 300 -268.311 , scale = -0.690135
step 400 -266.792 , scale = -0.587614
step 500 -265.506 , scale = -0.492658
step 600 -264.76 , scale = -0.418722
step 700 -264.515 , scale = -0.374801
step 800 -264.469 , scale = -0.355069
step 900 -264.464 , scale = -0.347956
step 1000 -264.463 , scale = -0.345793
step 1100 -264.463 , scale = -0.345226
step 1200 -264.463 , scale = -0.345101
step 1300 -264.462 , scale = -0.345077
step 1400 -264.462 , scale = -0.345071
step 1500 -264.462 , scale = -0.345071
step 1600 -264.462 , scale = -0.345071
step 1700 -264.462 , scale = -0.345071
step 1800 -264.462 , scale = -0.345071
step 1900 -264.462 , scale = -0.345071
step 2000 -264.462 , scale = -0.345071


In [10]:
plots = []
for i in range(probas.shape[1]):
    plots.append({'y': probas[0, i, :]})
    
iplot(plots)